In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchdiffeq import odeint
from scipy.integrate import solve_ivp

# 1. Sistema Dinâmico: Oscilador Harmônico Amortecido
damping_ratio = 0.1        # ζ: coef. amortecimento
natural_frequency = 2.0    # ω_n: frequência natural
t_max = 20.0               # tempo total simulação

# Solução analítica
def true_dynamics(t, state):
    dx1 = state[1] # Define que a derivada da posição (dx1) é a velocidade (state[1])
    dx2 = -2 * damping_ratio * natural_frequency * state[1] - natural_frequency**2 * state[0] #Define que a derivada da velocidade (dx2, ou seja, a aceleração) segue a equação clássica do oscilador amortecido.
    return [dx1, dx2] 

initial_state = [1.0, 0.0] #Define a condição inicial do sistema: começa na posição 1.0 e com velocidade 0.0

# 2. Definição da Rede Neural
#O objetivo desta rede não é prever a trajetória, mas sim aprender a função true_dynamics. Ela deve se tornar um substituto da equação física, aprendendo a prever a derivada (a taxa de mudança) para qualquer estado do sistema.
class ODEFunc(nn.Module): # Define uma classe que herda de nn.Module, a classe base para todos os modelos de rede neural em PyTorch
    def __init__(self): # O construtor da classe, onde as camadas da rede são definidas
        super(ODEFunc, self).__init__()  # Cria um contêiner sequencial para as camadas da rede.
        self.net = nn.Sequential(
            nn.Linear(2, 32),
            nn.Tanh(),
            nn.Linear(32, 2)
        )
    def forward(self, t, x): # Este é o método que executa a "passagem para frente" da rede. (tempo t e o estado x).
        return self.net(x)

# 3. Função de Experimento
# Esta é a função principal que encapsula todo o processo: gerar os dados, treinar a rede neural para aprender a dinâmica e, finalmente, testar a estabilidade da rede treinada usando diferentes métodos numéricos (solvers)
def run_experiment(num_points, methods, epochs=2000, lr=0.001, seed=55, save_prefix="exp"):
    """
    num_points : número de pontos da malha temporal
    methods    : lista de métodos numéricos a testar
    epochs     : número de épocas de treinamento
    lr         : taxa de aprendizado
    seed       : semente para reprodutibilidade
    """

    print(f"\n==== Rodando experimento com {num_points} pontos na malha (Seed: {seed}) ====")
    
    # Fixar a semente para garantir que cada treino comece igual
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Criar malha temporal
    t_eval = np.linspace(0, t_max, num_points)
    t_train = torch.tensor(t_eval, dtype=torch.float32)

    # Ground truth com SciPy (RK45 alta precisão)
    sol = solve_ivp(true_dynamics, [0, t_max], initial_state,
                    t_eval=t_eval, method='RK45', rtol=1e-10, atol=1e-12)
    ground_truth = sol.y.T
    x_train = torch.tensor(ground_truth, dtype=torch.float32)

    # Rede neural (agora inicializada de forma determinística)
    func = ODEFunc()
    optimizer = optim.Adam(func.parameters(), lr=lr) # Define o otimizador Adam, que será responsável por atualizar os pesos da rede com base no erro
    loss_fn = nn.MSELoss()

    # Treinamento
    print("Iniciando treinamento da ODEFunc...")
    for epoch in range(epochs):
        optimizer.zero_grad()
        # Usamos sempre um solver de alta qualidade para o TREINO
        pred_x = odeint(func, torch.tensor(initial_state, dtype=torch.float32), t_train, method="dopri5")
        loss = loss_fn(pred_x, x_train)
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")
    print("Treinamento concluído.")

    
    # Perturbação inicial
    perturbation = np.array([0.01, -0.01])
    initial_state_perturb = [initial_state[0] + perturbation[0],
                             initial_state[1] + perturbation[1]]

    # Ground truth perturbado
    sol_perturb = solve_ivp(true_dynamics, [0, t_max], initial_state_perturb,
                            t_eval=t_eval, method='RK45', rtol=1e-10, atol=1e-12)
    ground_truth_perturb = sol_perturb.y.T

    # Testar diferentes métodos no Neural ODE TREINADO
    predictions = {}
    predictions_perturb = {}
    
    print("Testando solvers na ODEFunc aprendida...")
    for m in methods:
        print(f"  -> Rodando método {m}")
        with torch.no_grad(): # Não precisamos de gradientes na fase de teste
            pred = odeint(func, torch.tensor(initial_state, dtype=torch.float32), t_train, method=m)
            pred_perturb = odeint(func, torch.tensor(initial_state_perturb, dtype=torch.float32), t_train, method=m)
        predictions[m] = pred.detach().numpy()
        predictions_perturb[m] = pred_perturb.detach().numpy()

    # Função norma de erro
    def error_norm(sol1, sol2):
        return np.linalg.norm(sol1 - sol2, axis=1)

    # Visualizações
    print("Gerando gráficos...")
    for m in methods:
        pred = predictions[m]
        pred_perturb = predictions_perturb[m]
        error_true = error_norm(ground_truth, ground_truth_perturb)
        error_neural = error_norm(pred, pred_perturb)

        plt.figure(figsize=(12, 5))

        # (a) Diagrama de fase
        plt.subplot(1, 2, 1)
        plt.plot(ground_truth[:, 0], ground_truth[:, 1], 'k-', label="Ground Truth (RK45)")
        plt.plot(pred[:, 0], pred[:, 1], 'r--', label=f"Neural ODE ({m})")
        plt.xlabel("x1 (posição)")
        plt.ylabel("x2 (velocidade)")
        plt.title(f"Diagrama de Fase - Malha N={num_points}, Solver {m}")
        plt.legend()

        # (b) Estabilidade via Lyapunov
        plt.subplot(1, 2, 2)
        plt.plot(t_eval, error_true, 'k-', label="Erro Ground Truth (perturbado)")
        plt.plot(t_eval, error_neural, 'r--', label=f"Erro Neural ODE ({m})")
        plt.xlabel("Tempo")
        plt.ylabel("||x - x_p||")
        plt.title(f"Estabilidade - Malha N={num_points}, Solver {m}")
        plt.legend()

        plt.tight_layout()
        plt.savefig(f"{save_prefix}_N{num_points}_{m}.png")
        plt.close()

# 5. Execução dos Experimentos
methods_to_test = ["euler", "rk4", "dopri5"]
shared_seed = 2 # Semente compartilhada para todos os experimentos

# Testar para 3 malhas diferentes, todas começando com os mesmos pesos iniciais
for num_points in [100]:
    run_experiment(num_points, methods_to_test, epochs=2000, seed=shared_seed, save_prefix="bench_controlled")


==== Rodando experimento com 100 pontos na malha (Seed: 2) ====
Iniciando treinamento da ODEFunc...
Epoch 0, Loss: 0.374782
Epoch 500, Loss: 0.089518
Epoch 1000, Loss: 0.003517
Epoch 1500, Loss: 0.001746
Treinamento concluído.
Testando solvers na ODEFunc aprendida...
  -> Rodando método euler
  -> Rodando método rk4
  -> Rodando método dopri5
  -> Rodando método explicit_adams
Gerando gráficos...
